# December 19, 2022
https://adventofcode.com/2022/day/19

In [2]:
from math import ceil
import re

In [3]:
def create_blueprint( orebot, claybot, obsbot_ore, obsbot_clay, geobot_ore, geobot_obs ):
    return {
        'orebot': {'ore': orebot},
        'claybot': {'ore': claybot},
        'obsbot': {'ore': obsbot_ore, 'clay': obsbot_clay},
        'geobot': {'ore': geobot_ore, 'obs': geobot_obs}
        }

test_bps = [
    create_blueprint( 4, 2, 3,14,  2,7 ),
    create_blueprint( 2, 3,  3,8, 3,12 )
]

In [4]:
fn = "../data/2022/19.txt"
puz_bps = []

with open(fn, "r") as file:
    while True:
        line = file.readline().strip("\n")
        if not line:
            break
        ma = re.search( r"\d+.*(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+", line )
        specs = []
        for i in range(1,7):
            specs.append(int(ma.group(i)))
        puz_bps.append( create_blueprint( *specs ) ) 

In [36]:
def robot_factory( time, bp, memoize = False, orebot=1, ore=0, claybot=0, clay=0, obsbot=0, obs=0, geobot=0  ):
    '''This algo caps orebots at 3 so we don't waste time trying bad strats. This is not 100% guaranteed!'''

    # No point in getting more bots than this because you can't spend it any faster.
    max_orebot = max( bp['orebot']['ore'], bp['claybot']['ore'], bp['obsbot']['ore'], bp['geobot']['ore'] )
    max_claybot = bp['obsbot']['clay']
    max_obsbot = bp['geobot']['obs']

    global memo
    if memoize:
        key = ".".join([str(x) for x in [time,orebot,ore,claybot,clay,obsbot,obs]])
        if key in memo:
            return memo[key]

    if time < 1:
        return 0

    # Once we start making geobots, we probably are past making orebots and claybots:
    if geobot > 0:
        max_orebot = orebot
        max_claybot = claybot

        # try another obsbot if it will possibly lead to another geobot
        # This heuristic may lead to wrong answer. Don't use it
        #pred_geobot = int( bp['geobot']['obs'] / (time*obsbot + obs) )
        #poss_geobot = int( bp['geobot']['obs'] / (time*obsbot + obs + (time-1)) )
        #if poss_geobot <= pred_geobot:
        #    max_obsbot = obsbot

   # print(time, orebot, ore, claybot, clay, obsbot, obs)

    # Each branching point has four options: what to build next?
    best = 0

    ## Option 1: Build an Orebot ##
    if orebot < max_orebot:
        ore_req = bp['orebot']['ore']
        ore_needed = max(ore_req-ore, 0)
        time_needed = 1 + ceil( ore_needed / orebot ) # 1 time to build bot. ceil(.) may be 0 if we already have enough ore

        # Is there enough time to make this orebot?
        # using strict because we need time to make the robot and also time for it to do a thing
        if time_needed < time:
            # How much ore is gained while we wait?
            # We may produce extra ore. e.g. if we need 1 but have 3 orebots.
            # Actually, we always produce extra ore because the orebot continues collecting while we build
            ore_gained = time_needed * orebot 

            # After waiting, we've gained some ore and spent some ore
            ore_left = ore + ore_gained - ore_req
            # We've also lost some time to collect ore and build the orebot
            time_left = time - time_needed

            # Plus we may gain other resources while we wait for ore
            clay_left = clay + claybot * time_needed
            obs_left = obs + obsbot * time_needed

            # Iterate, but we have one more orebot, and a different amount of ore/clay/obs/time
            gain = robot_factory( time_left, bp, memoize, orebot+1, ore_left, claybot, clay_left, obsbot, obs_left, geobot)
            best = max( best, gain )

    ## Option 2: Build a Claybot ##
    if claybot < max_claybot:
        ore_req = bp['claybot']['ore']
        ore_needed = max(ore_req-ore, 0)
        time_needed = 1 + ceil( ore_needed / orebot ) 

        if time_needed < time:
            ore_gained = time_needed * orebot 
            ore_left = ore + ore_gained - ore_req
            time_left = time - time_needed

            clay_left = clay + claybot * time_needed
            obs_left = obs + obsbot * time_needed

            gain = robot_factory( time_left, bp, memoize, orebot, ore_left, claybot+1, clay_left, obsbot, obs_left, geobot )
            best = max( best, gain )

    ## Option 3: Build an Obsidianbot ##
    if claybot > 0 and obsbot < max_obsbot:
        ore_req = bp['obsbot']['ore']
        clay_req = bp['obsbot']['clay']

        ore_needed = max(ore_req-ore, 0)
        clay_needed = max(clay_req-clay, 0)

        time_needed = 1 + max( ceil(ore_needed/orebot), ceil(clay_needed/claybot) )

        if time_needed < time:
            ore_left  =  ore + (time_needed *  orebot) -  ore_req
            clay_left = clay + (time_needed * claybot) - clay_req
            time_left = time - time_needed
            obs_left = obs + obsbot * time_needed

            gain = robot_factory( time_left, bp, memoize, orebot, ore_left, claybot, clay_left, obsbot+1, obs_left, geobot )
            best = max(best, gain)

    ## Option 4: Build a Geodebot ##
    if obsbot > 0:
        ore_req = bp['geobot']['ore']
        obs_req = bp['geobot']['obs']

        ore_needed = max(ore_req-ore, 0)
        obs_needed = max(obs_req-obs, 0)

        time_needed = 1 + max( ceil(ore_needed/orebot), ceil(obs_needed/obsbot) )

        if time_needed < time:
            ore_left = ore + (time_needed * orebot) - ore_req
            obs_left = obs + (time_needed * obsbot) - obs_req
            clay_left = clay + claybot * time_needed
            time_left = time - time_needed

            direct_gain = time_left # Gain 1 geode per time
            gain = direct_gain + robot_factory( time_left, bp, memoize, orebot, ore_left, claybot, clay_left, obsbot, obs_left, geobot+1 )
            best = max(best, gain)

    if memoize:
        memo[key] = best
    return best

### Part 1

In [37]:
# 5s without memo
# 2.5s with memo
memo = {}
robot_factory(24, test_bps[0], memoize=True)

9

In [38]:
# ~3s without memo
# 2s with memo
memo = {}
robot_factory(24, test_bps[1], memoize=True)

12

In [39]:
# ~16s without memo
# ~ 11s with memo
out = []
# I think some of these bps don't let you get any geodes...
for bp in puz_bps:
    memo = {}
    ans = robot_factory(24, bp, memoize=True)
    out.append(ans)

print( ", ".join([str(x) for x in out]))

part1 = sum( [(i+1)*x for i,x in enumerate(out)] )
print("Part1:", part1)

4, 0, 12, 1, 0, 7, 3, 0, 1, 0, 9, 0, 1, 0, 2, 3, 12, 1, 2, 3, 13, 10, 2, 0, 1, 1, 2, 4, 0, 0
Part1: 1382


With orebot capped at 3:
'4, 0, 12, 0, 0, 7, 2, 0, 1, 0, 8, 0, 1, 0, 2, 3, 12, 1, 1, 3, 13, 10, 1, 0, 0, 0, 2, 4, 0, 0'

With orebot capped at 6:
'4, 0, 12, 1, 0, 7, 3, 0, 1, 0, 9, 0, 1, 0, 2, 3, 12, 1, 2, 3, 13, 10, 2, 0, 1, 1, 2, 4, 0, 0'

With all bots capped at max usage:
'4, 0, 12, 1, 0, 7, 3, 0, 1, 0, 9, 0, 1, 0, 2, 3, 12, 1, 2, 3, 13, 10, 2, 0, 1, 1, 2, 4, 0, 0'

### Part 2

In [43]:
memo = {}
robot_factory(24, test_bps[0], memoize=True)


9

In [44]:
memo = {}
robot_factory(24, test_bps[0], memoize=True)


9

In [45]:
memo = {}
robot_factory(25, test_bps[0], memoize=True)


12

In [46]:
memo = {}
robot_factory(26, test_bps[0], memoize=True)


15

In [47]:
memo = {}
robot_factory(27, test_bps[0], memoize=True)


20

In [48]:
memo = {}
robot_factory(28, test_bps[0], memoize=True)


26

In [49]:
memo = {}
robot_factory(29, test_bps[0], memoize=True)


32

In [50]:
# Without memo... over 11minutes
memo = {}
robot_factory(30, test_bps[0], memoize=True)

39

In [51]:
# Without memo... over 11minutes
memo = {}
robot_factory(31, test_bps[0], memoize=True)

47

In [53]:
memo = {}
robot_factory(32, test_bps[0], memoize=True)

56

In [56]:
memo = {}
robot_factory(32, test_bps[1], memoize=True)

62

In [61]:
out = []
# I think some of these bps don't let you get any geodes...
for bp in puz_bps[:3]:
    memo = {}
    ans = robot_factory(32, bp, memoize=True)
    out.append(ans)
    
print(", ".join([str(x) for x in out]))
part2 = out[0]*out[1]*out[2]
print(part2)

46, 10, 69
31740
